In [0]:
import geopandas as gpd
from shapely.geometry.base import BaseGeometry
from shapely import force_2d
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    TimestampType,
    IntegerType,
)

In [0]:
gcs_landing_zone = "/Volumes/land_auto-gen-kart_dev/external_dev/landing_zone"
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "hospitals_bronze"
log_table = "logs_processed_hospital_gdbs"
layer = "BygningsPunkt_Sykehus_med_akuttmottak"

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {bronze_table} (
    row_hash STRING,
    source_file STRING,
    source_layer STRING,
    geometry BINARY,
    ingest_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {log_table} (
  gdb_name STRING,
  processed_time TIMESTAMP,
  num_inserted INT,
  num_updated INT,
  num_deleted INT
) USING DELTA
"""
spark.sql(q)

In [0]:
def log_processed_gdb(log_data: list):
    """
    Skriver logg med antall insert, update og deleter i deltatabellen.
    """
    schema = StructType(
        [
            StructField("gdb_name", StringType(), True),
            StructField("processed_time", TimestampType(), True),
            StructField("num_inserted", IntegerType(), True),
            StructField("num_updated", IntegerType(), True),
            StructField("num_deleted", IntegerType(), True),
        ]
    )
    spark.createDataFrame(log_data, schema=schema).write.format("delta").mode(
        "append"
    ).saveAsTable(log_table)

In [0]:
def check_for_new_gdbs() -> list:
    """
    Returnerer en liste med geodatabaser som ikke er lagret i deltatabellen.
    """
    all_gdbs = [
        f.path for f in dbutils.fs.ls(gcs_landing_zone) if f.path.endswith("Sykehus.gdb/")
    ]
    processed_gdbs_df = spark.read.table(log_table).select("gdb_name")
    processed_gdbs = [row["gdb_name"] for row in processed_gdbs_df.collect()]

    return [gdb for gdb in all_gdbs if gdb not in processed_gdbs]

In [0]:
def to_wkt_2d(geom):
    """
    Konverterer geometri til WKT format i 2D format.
    """
    if isinstance(geom, BaseGeometry):
        return force_2d(geom).wkt
    return None

In [0]:
def write_to_sdf(gdb_path: str, gdb_name: str, layer: str) -> DataFrame:
    """
    Returnerer en spark dataframe med data fra deltatabellen.
    """
    gdf = (
        gpd.read_file(gdb_path, layer=layer).set_crs("EPSG:25833").to_crs("EPSG:25833")
    )
    gdf["wkt_geometry"] = gdf["geometry"].apply(to_wkt_2d)
    gdf = gdf.drop(columns=["geometry"])

    sdf = spark.createDataFrame(gdf)
    sdf = sdf.withColumnRenamed("wkt_geometry", "geometry")
    sdf = (
        sdf.withColumn("ingest_time", current_timestamp())
        .withColumn("source_file", lit(gdb_name))
        .withColumn("source_layer", lit(layer))
        .withColumn("row_hash", sha2(concat_ws("||", *sdf.columns), 256))
    )

    target_cols = DeltaTable.forName(spark, bronze_table).toDF().columns
    sdf = sdf.select([c for c in sdf.columns if c in target_cols])

    return sdf

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og oppdaterer dersom row_hash allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(bronze_table)
    else:
        from delta.tables import DeltaTable

        delta_tbl = DeltaTable.forName(spark, bronze_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.row_hash = source.row_hash"
        ).whenMatchedUpdate(
            condition="target.ingest_time < source.ingest_time",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

In [0]:
def write_to_delta_table(sdf: DataFrame, gdb_name: str):
    """
    Skriver logg med antall insert, update og deleter i deltatabellen og lagrer denne.
    """
    table_exists = False
    if spark.catalog.tableExists(bronze_table):
        delta_tbl = DeltaTable.forName(spark, bronze_table)
        version_before = delta_tbl.history(1).select("version").collect()[0][0]
        table_exists = True

    write_delta_table(sdf)

    if table_exists:
        version_after = delta_tbl.history(1).select("version").collect()[0][0]
        if version_after > version_before:
            metrics = delta_tbl.history(1).select("operationMetrics").collect()[0][0]
            updated = int(metrics.get("numTargetRowsUpdated", 0))
            inserted = int(metrics.get("numTargetRowsInserted", 0))
            deleted = int(metrics.get("numTargetRowsDeleted", 0))
            print(f"Updated: {updated}, Inserted: {inserted}, Deleted: {deleted}")
        else:
            print("No new Delta version found after merge.")
    else:
        inserted, updated, deleted = sdf.count(), 0, 0
        print(f"Updated: {updated}, Inserted: {inserted}, Deleted: {deleted}")

    log_processed_gdb(log_data=[(gdb_name, datetime.now(), inserted, updated, deleted)])

In [0]:
def main():
    """
    Finner nye geodatabaser og skriver til deltatabellen.
    """
    gdbs = check_for_new_gdbs()
    for gdb in gdbs:
        gdb_name = gdb.rstrip("/").split("/")[-1]
        gdb_path = gdb.removeprefix("dbfs:")
        print(f"\nProcessing gdb: {gdb_name}")

        sdf = write_to_sdf(gdb_path, gdb_name, layer)
        write_to_delta_table(sdf, gdb_name)

In [0]:
main()